# DarkSky: Scraping weather data 

The aim of this project is to scrape weather data from multiple URLs and save it in as CSV,as tables MySQL and as collection in MongoDB.

The Dark Sky API (https://darksky.net/dev) allows you to look up the weather anywhere on the globe.You can make 2 kinds of API calls; Forecast request and Time Machine Request.

## Import necessary Libraries

In [1]:
## Beloved Pandas
import pandas as pd

## Libraries needed to connect to URL and request for data 
import requests
from pandas.io.json import json_normalize #Normalize semi-structured JSON data into a flat table.
 
## Libraries needed to work with time
import time
import datetime
from datetime import date, timedelta

# Libraries needed to connect to MySQL
from sqlalchemy import create_engine
import pymysql

# Libraries needed to connect to MongoDB
import pymongo
from pymongo import MongoClient

## Setting up intial files

Read the file with store ID(105 IDs with unique latitude and longitude). Isolate all 3 columns to use them individually later. 

Base URL contails the API key and can be used to create list of URLs to get Future predicted data and Historical weather data.

In [2]:
#Read excel file
location=pd.read_csv(r'C:\Users\megha\Documents\Projects\DarkSky\store.csv') #105 store  IDs

#API Key
#authorization_code='ffb7c6d0d32876c01496218fbd979218'

#isolate latitude and longitude
lat,long=location['LATITUDE'],location['LONGITUDE'] #needed to create URls
store_id=location['LOC_ID'] 

#base url
base_url="https://api.darksky.net/forecast/ffb7c6d0d32876c01496218fbd979218/"

##  Time Machine request 

Enter start date and end date for the which you need weather data and creat a list of dates, convert it to unix time as API calls are made using URLs with location and unixtime. Use the date in unix time, Location and base URL to create a list of URL you need to request to get the data. The data is in Json format and has to be normalized to a flat table to be converted to Dataframe.

In [ ]:
#Past date weather
#Get time for Time_machine date
dates=[]

start_date = date(2019, 12, 1)
end_date = date(2019, 12, 5)
delta = timedelta(days=1)

while start_date <= end_date:
    c=start_date.strftime("%d/%m/%Y")
    dates.append(c)
    start_date += delta
date_unix=[]
for i in dates:
    f=time.mktime(datetime.datetime.strptime(i, "%d/%m/%Y").timetuple())
    date_unix.append(f)
    
#make list of URLs    
url=''
for i,j in zip(lat,long):
    for k in date_unix:
        url +=base_url+str(i)+str(',')+str(j)+str(',')+str(int(k))+str(" ")
    url_list=url.split(" ")
    url_list.remove('')
    
#Dataframe with old weather data
Time_machine=pd.DataFrame()
for url in url_list:
    response=requests.get(url)
    r=response.json()
    df=json_normalize(r['currently'])
    df['LATITUDE']=r['latitude']
    df['LONGITUDE']=r['longitude']
    Time_machine= Time_machine.append(df, ignore_index=True)
    Time_machine['date'] = pd.to_datetime(Time_machine['time'],unit='s')

Time_machine_data=pd.merge(location, Time_machine, on=['LATITUDE', 'LONGITUDE'])

C:\Users\megha\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


## Forecast Request

Create a list of URL using only location and base URL, forecast does not need time. Use the URL list to create multiple requests. The data is in Json format and has to be normalized to a flat table to be converted to Dataframe.

In [ ]:
# forecastorecast forecastor forecastututre
#making a list oforecast URLs
url_forecast=''
for i,j in zip(lat,long):
    url_forecast +=base_url+str(i)+str(',')+str(j)+str(" ")
    url_list_forecast=url_forecast.split(" ")
    url_list_forecast.remove('')

#Dataforecastrame with new prediction data
forecast=pd.Dataforecastrame()
for url,i in zip(url_list_forecast,store_id):
    response=requests.get(url)
    r=response.json()
    data=json_normalize(r,['daily','data'],['latitude','longitude'])
    data['date'] = pd.to_datetime(data['time'],unit='s')
    data['store_id']=i
    forecast_data = forecast.append(data, ignore_index=True)


## CSV

In [ ]:
Time_machine_data.to_csv('History.csv')
forecast_data.to_csv('Forecasting.csv')

## MySQL

In [ ]:
#https://docs.sqlalchemy.org/en/13/core/engines.html
#https://pythontic.com/pandas/serialization/mysql
#moving both dataframes to MySQL database
table1='DarkSkyForecast'
table2='DarkSkyHistory'

#create this schema in SQL prior to running program
tableName='darksky'

sqlEngine = create_engine('mysql+pymysql://root:megh@n@1811@localhost/darksky', pool_recycle=3600)

dbConnection = sqlEngine.connect() 
try:
    frameh = Time_machine_data.to_sql(table2, dbConnection, if_exists='fail');
    framef = forecast_data.to_sql(table1, dbConnection, if_exists='fail')
except ValueError as vx:
    print(vx)
except Exception as ex:   
    print(ex)
else:
    print("Table %s created successfully."%tableName);   
finally:
    dbConnection.close()

## MongoDB

In [ ]:
#https://stackoverflow.com/questions/20167194/insert-a-pandas-dataframe-into-mongodb-using-pymongo
client = MongoClient('localhost', 27017)
#or use
#client = MongoClient('mongodb://localhost:27017')
db=client['Weather_data']
collection_1=db['Forecast']
collection_2=db['History']

collection_1.insert_many(Time_machine_data.to_dict('record'))
collection_2.insert_many(forecast_data.to_dict('record'))